# Lesson 3: Multi-Agent Systems with Hierarchical Routing

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:

1. **Understand** why and when to use multi-agent systems
2. **Implement** the hierarchical routing pattern (coordinator + specialists)
3. **Create** specialized agents with domain-specific tools and expertise
4. **Build** a coordinator agent that intelligently routes requests
5. **Demonstrate** how agents communicate and transfer control
6. **Apply** this pattern to real-world production scenarios

## 📚 Quick Recap: Lessons 1-2

In previous lessons, you learned:
- ✅ How to create basic agents with personality and instructions
- ✅ How to give agents tools (function calling)
- ✅ How to manage sessions and conversation context

**Limitation**: A single agent with many tools becomes complex, slow, and hard to maintain.

## 🚀 What's New in This Lesson?

We're going to build a **multi-agent system** using the **hierarchical routing pattern** - the most important design pattern for production AI systems. Instead of one agent doing everything, we'll create:

- 🎯 **Coordinator Agent**: Routes requests to the right specialist
- 🔧 **Hardware Specialist**: Handles hardware issues with specialized tools
- 💻 **Software Specialist**: Handles software issues with specialized tools
- 🌐 **Network Specialist**: Handles network issues with specialized tools

## 🏢 Use Case: Enterprise IT Support System

We're building a production-grade IT support system where different types of issues are handled by specialized agents with deep expertise in their domains.

---

## 💡 Part 1: Why Multi-Agent Systems?

### The Problem with Single-Agent Approach

Imagine one IT support agent with 50+ tools:
- ❌ **Overwhelming context**: Agent must understand all tools at once
- ❌ **Slow responses**: More tools = more processing time
- ❌ **Generic responses**: Jack of all trades, master of none
- ❌ **Maintenance nightmare**: Updating one tool affects everything
- ❌ **Cost inefficiency**: Using expensive models for simple tasks

### The Multi-Agent Solution

```
                    ┌─────────────────┐
    User Request    │   Coordinator   │  (Routes to specialist)
    ────────────>   │     Agent       │
                    └────────┬────────┘
                             │
              ┌──────────────┼──────────────┐
              ▼              ▼              ▼
        ┌──────────┐   ┌──────────┐   ┌──────────┐
        │ Hardware │   │ Software │   │ Network  │
        │Specialist│   │Specialist│   │Specialist│
        └──────────┘   └──────────┘   └──────────┘
        3 tools        3 tools        3 tools
```

### Benefits of Hierarchical Routing

✅ **Specialization**: Each agent is an expert in their domain
✅ **Faster responses**: Specialists only load relevant tools
✅ **Better quality**: Focused instructions for each domain
✅ **Easy maintenance**: Update one specialist without affecting others
✅ **Scalability**: Add new specialists without rebuilding everything
✅ **Cost optimization**: Use different models for different complexity levels
✅ **Clear debugging**: Know exactly which agent handled each request

### Real-World Applications

This pattern is used in production by:
- **Customer support**: Sales, technical, billing specialists
- **Healthcare**: Diagnosis, treatment, prescription specialists
- **Finance**: Investment, risk, compliance specialists
- **E-commerce**: Product, shipping, returns specialists

**This is THE pattern you need to know for building production AI systems.**

---

## 🔧 Part 2: Environment Setup

Let's install packages and set up our environment.

In [ ]:
# Install the Google Agent Development Kit and dependencies
!pip install -q google-adk litellm openai python-dotenv nest-asyncio

print("✅ Packages installed successfully!")

In [ ]:
# Core ADK imports
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm
from google.genai import types

# System imports
import os
import asyncio
from typing import Dict, List, Optional
from datetime import datetime
import random

print("✅ Imports successful!")

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Configure which OpenAI model to use
# Using gpt-5-nano for all agents (most cost-efficient for learning)
OPENAI_MODEL = "gpt-5-nano"  # Default model for this tutorial

print(f"🤖 Selected Model: {OPENAI_MODEL}")
print("\n💡 Model Selection Notes:")
print("   - This tutorial uses gpt-5-nano for cost efficiency")
print("   - In production, you might use more powerful models for complex routing:")
print("     • Coordinator: gpt-4o (better at complex routing decisions)")
print("     • Specialists: gpt-5-nano or gpt-4o-mini (routine tasks)")
print("\n💡 ADK is model-agnostic! Same code works with:")
print("   - OpenAI: model='gpt-5-nano', 'gpt-4o-mini', 'gpt-4o'")
print("   - Claude: model='claude-3-5-sonnet-20241022'")
print("   - Gemini: model='gemini-2.0-flash-exp'")

---

## 🛠️ Part 3: Creating Specialist Agents

Let's create three specialist agents, each with their own domain-specific tools.

### Design Principle: Separation of Concerns

Each specialist:
- Has **focused expertise** in one domain
- Uses **specialized tools** relevant to their area
- Has **tailored instructions** for their specific role
- Maintains **independent context** (what they need to know)

### 3.1: Hardware Specialist Tools

In [ ]:
# Mock hardware database
HARDWARE_WARRANTY_DB = {
    "laptop": {"warranty_end": "2026-12-31", "coverage": "full", "manufacturer": "Dell"},
    "desktop": {"warranty_end": "2025-06-30", "coverage": "parts_only", "manufacturer": "HP"},
    "monitor": {"warranty_end": "2024-01-15", "coverage": "expired", "manufacturer": "Samsung"},
    "printer": {"warranty_end": "2027-03-20", "coverage": "full", "manufacturer": "Canon"},
}

def check_warranty(device_type: str) -> Dict[str, any]:
    """
    Checks the warranty status for a hardware device.
    
    Use this tool to verify if a device is still under warranty and what coverage it has.
    This helps determine if repairs should be covered by warranty or require purchase approval.
    
    Args:
        device_type (str): Type of device to check (e.g., 'laptop', 'desktop', 'monitor', 'printer')
    
    Returns:
        Dict: Warranty information including expiration date, coverage type, and manufacturer
    """
    print(f"🔧 [HARDWARE TOOL] check_warranty(device_type='{device_type}')")
    
    device_type_lower = device_type.lower().strip()
    
    if device_type_lower in HARDWARE_WARRANTY_DB:
        warranty_info = HARDWARE_WARRANTY_DB[device_type_lower]
        return {
            "success": True,
            "device_type": device_type_lower,
            "warranty_end": warranty_info["warranty_end"],
            "coverage": warranty_info["coverage"],
            "manufacturer": warranty_info["manufacturer"],
            "is_covered": warranty_info["coverage"] != "expired"
        }
    else:
        return {
            "success": False,
            "error": f"Device type '{device_type}' not found in warranty database",
            "available_types": list(HARDWARE_WARRANTY_DB.keys())
        }


def run_hardware_diagnostic(device_type: str) -> Dict[str, any]:
    """
    Runs a hardware diagnostic test on a device.
    
    Use this tool to identify hardware issues like failing components, overheating,
    or connection problems. The diagnostic simulates real hardware testing.
    
    Args:
        device_type (str): Type of device to diagnose (e.g., 'laptop', 'desktop', 'printer')
    
    Returns:
        Dict: Diagnostic results with status, detected issues, and recommendations
    """
    print(f"🔧 [HARDWARE TOOL] run_hardware_diagnostic(device_type='{device_type}')")
    
    # Simulate diagnostic test
    possible_issues = [
        {"component": "Hard Drive", "status": "Warning", "message": "High temperature detected"},
        {"component": "RAM", "status": "Pass", "message": "All memory tests passed"},
        {"component": "CPU", "status": "Pass", "message": "Operating within normal parameters"},
        {"component": "Power Supply", "status": "Fail", "message": "Voltage fluctuations detected"},
        {"component": "Graphics Card", "status": "Pass", "message": "No issues detected"},
    ]
    
    # Randomly select 2-4 diagnostic results
    num_results = random.randint(2, 4)
    diagnostic_results = random.sample(possible_issues, num_results)
    
    has_issues = any(result["status"] in ["Fail", "Warning"] for result in diagnostic_results)
    
    return {
        "success": True,
        "device_type": device_type,
        "overall_status": "Issues Detected" if has_issues else "All Tests Passed",
        "diagnostic_results": diagnostic_results,
        "recommendation": "Schedule repair" if has_issues else "No action needed"
    }


def order_replacement_part(device_type: str, part_name: str) -> Dict[str, any]:
    """
    Orders a replacement hardware part.
    
    Use this tool when diagnostic tests reveal a failed component that needs replacement.
    This simulates ordering parts from the IT inventory system.
    
    Args:
        device_type (str): Type of device needing repair (e.g., 'laptop', 'desktop')
        part_name (str): Name of the part to order (e.g., 'Hard Drive', 'Power Supply')
    
    Returns:
        Dict: Order confirmation with tracking number and estimated delivery
    """
    print(f"🔧 [HARDWARE TOOL] order_replacement_part(device_type='{device_type}', part_name='{part_name}')")
    
    # Simulate order creation
    order_number = f"HW-{random.randint(10000, 99999)}"
    delivery_days = random.randint(2, 5)
    
    return {
        "success": True,
        "order_number": order_number,
        "device_type": device_type,
        "part_name": part_name,
        "estimated_delivery": f"{delivery_days} business days",
        "status": "Order placed",
        "message": f"Replacement {part_name} ordered for {device_type}. You will receive email updates."
    }

print("✅ Hardware specialist tools created!")

### 3.2: Software Specialist Tools

In [ ]:
# Mock software license database
SOFTWARE_LICENSE_DB = {
    "microsoft office": {"status": "active", "expires": "2026-01-15", "seats_used": 245, "seats_total": 300},
    "adobe creative cloud": {"status": "active", "expires": "2025-12-31", "seats_used": 50, "seats_total": 50},
    "zoom": {"status": "active", "expires": "2026-06-30", "seats_used": 180, "seats_total": 200},
    "antivirus": {"status": "expired", "expires": "2024-11-30", "seats_used": 300, "seats_total": 300},
}

def check_license(software_name: str) -> Dict[str, any]:
    """
    Checks the license status for a software application.
    
    Use this tool to verify if a software license is active, how many seats are available,
    and when it expires. This helps troubleshoot activation issues.
    
    Args:
        software_name (str): Name of the software to check (e.g., 'Microsoft Office', 'Adobe Creative Cloud')
    
    Returns:
        Dict: License information including status, expiration, and seat availability
    """
    print(f"💻 [SOFTWARE TOOL] check_license(software_name='{software_name}')")
    
    software_lower = software_name.lower().strip()
    
    if software_lower in SOFTWARE_LICENSE_DB:
        license_info = SOFTWARE_LICENSE_DB[software_lower]
        seats_available = license_info["seats_total"] - license_info["seats_used"]
        
        return {
            "success": True,
            "software_name": software_name,
            "status": license_info["status"],
            "expires": license_info["expires"],
            "seats_available": seats_available,
            "seats_total": license_info["seats_total"],
            "can_activate": license_info["status"] == "active" and seats_available > 0
        }
    else:
        return {
            "success": False,
            "error": f"Software '{software_name}' not found in license database",
            "available_software": list(SOFTWARE_LICENSE_DB.keys())
        }


def reinstall_software(software_name: str) -> Dict[str, any]:
    """
    Reinstalls a software application to fix corruption or configuration issues.
    
    Use this tool when software is crashing, not opening, or behaving incorrectly.
    This simulates a clean reinstallation process.
    
    Args:
        software_name (str): Name of the software to reinstall (e.g., 'Microsoft Office')
    
    Returns:
        Dict: Reinstallation results with status and next steps
    """
    print(f"💻 [SOFTWARE TOOL] reinstall_software(software_name='{software_name}')")
    
    # Simulate reinstallation
    steps_completed = [
        "Uninstalled previous version",
        "Downloaded latest version",
        "Installed software",
        "Applied configuration",
        "Verified installation"
    ]
    
    return {
        "success": True,
        "software_name": software_name,
        "status": "Reinstallation completed",
        "steps_completed": steps_completed,
        "message": f"{software_name} has been successfully reinstalled. Please restart your computer.",
        "next_steps": ["Restart computer", "Test software functionality", "Restore user preferences"]
    }


def update_application(software_name: str) -> Dict[str, any]:
    """
    Updates a software application to the latest version.
    
    Use this tool when software is outdated, missing features, or has known bugs
    that are fixed in newer versions.
    
    Args:
        software_name (str): Name of the software to update
    
    Returns:
        Dict: Update results with version information and changes
    """
    print(f"💻 [SOFTWARE TOOL] update_application(software_name='{software_name}')")
    
    # Simulate version update
    old_version = f"{random.randint(1, 5)}.{random.randint(0, 9)}.{random.randint(0, 99)}"
    new_version = f"{random.randint(1, 5)}.{random.randint(0, 9)}.{random.randint(0, 99)}"
    
    return {
        "success": True,
        "software_name": software_name,
        "old_version": old_version,
        "new_version": new_version,
        "status": "Update completed",
        "changes": [
            "Security patches applied",
            "Performance improvements",
            "Bug fixes included",
            "New features added"
        ],
        "message": f"{software_name} updated from v{old_version} to v{new_version}"
    }

print("✅ Software specialist tools created!")

### 3.3: Network Specialist Tools

In [ ]:
def ping_host(hostname: str) -> Dict[str, any]:
    """
    Pings a network host to check connectivity.
    
    Use this tool to verify if a device or server is reachable on the network.
    This helps diagnose connection issues and network outages.
    
    Args:
        hostname (str): Hostname or IP address to ping (e.g., 'google.com', '192.168.1.1')
    
    Returns:
        Dict: Ping results with response time and packet loss
    """
    print(f"🌐 [NETWORK TOOL] ping_host(hostname='{hostname}')")
    
    # Simulate ping test
    is_reachable = random.random() > 0.2  # 80% success rate
    
    if is_reachable:
        response_time = random.randint(5, 150)
        packet_loss = random.randint(0, 5)
        
        return {
            "success": True,
            "hostname": hostname,
            "reachable": True,
            "response_time_ms": response_time,
            "packet_loss_percent": packet_loss,
            "status": "Connected" if packet_loss < 2 else "Unstable connection",
            "message": f"Host {hostname} is reachable with {response_time}ms response time"
        }
    else:
        return {
            "success": True,
            "hostname": hostname,
            "reachable": False,
            "status": "Connection failed",
            "message": f"Unable to reach {hostname}. Check network connection or firewall settings."
        }


def check_bandwidth(location: str) -> Dict[str, any]:
    """
    Checks network bandwidth and speed at a specific location.
    
    Use this tool to diagnose slow network performance or bandwidth issues.
    Helps identify if connection problems are due to insufficient bandwidth.
    
    Args:
        location (str): Network location to test (e.g., 'Building A', 'Lab 3', 'Office 201')
    
    Returns:
        Dict: Bandwidth test results with download/upload speeds and latency
    """
    print(f"🌐 [NETWORK TOOL] check_bandwidth(location='{location}')")
    
    # Simulate bandwidth test
    download_speed = random.randint(50, 500)  # Mbps
    upload_speed = random.randint(20, 100)    # Mbps
    latency = random.randint(5, 50)           # ms
    
    # Determine quality
    if download_speed > 200 and latency < 20:
        quality = "Excellent"
    elif download_speed > 100 and latency < 35:
        quality = "Good"
    else:
        quality = "Poor - May need optimization"
    
    return {
        "success": True,
        "location": location,
        "download_speed_mbps": download_speed,
        "upload_speed_mbps": upload_speed,
        "latency_ms": latency,
        "quality": quality,
        "message": f"Bandwidth at {location}: {download_speed} Mbps down / {upload_speed} Mbps up"
    }


def reset_router(location: str) -> Dict[str, any]:
    """
    Resets a network router to resolve connectivity issues.
    
    Use this tool when users report network outages, slow connections, or
    intermittent connectivity. Router reset often resolves these issues.
    
    Args:
        location (str): Location of the router to reset (e.g., 'Building A Floor 2')
    
    Returns:
        Dict: Reset results with status and estimated downtime
    """
    print(f"🌐 [NETWORK TOOL] reset_router(location='{location}')")
    
    # Simulate router reset
    return {
        "success": True,
        "location": location,
        "status": "Router reset completed",
        "downtime_seconds": 45,
        "steps_completed": [
            "Initiated router reboot",
            "Cleared connection table",
            "Renewed DHCP leases",
            "Restored network services"
        ],
        "message": f"Router at {location} has been reset. Network connectivity should be restored.",
        "next_steps": "If issues persist, check cable connections and contact network team."
    }

print("✅ Network specialist tools created!")

### 3.4: Create the Specialist Agents

Now let's create three specialist agents, each with their domain-specific tools and expertise.

In [ ]:
# Create Hardware Specialist Agent
hardware_specialist = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="hardware_specialist",
    tools=[
        check_warranty,
        run_hardware_diagnostic,
        order_replacement_part
    ],
    instruction="""
    You are a Hardware Specialist for TechHelp Solutions IT support.
    
    YOUR EXPERTISE:
    - Computer hardware (laptops, desktops, monitors, printers)
    - Hardware diagnostics and troubleshooting
    - Warranty management and part replacement
    - Physical device issues (power, boot, connection problems)
    
    YOUR TOOLS:
    - check_warranty: Verify warranty status before recommending repairs
    - run_hardware_diagnostic: Test hardware to identify failing components
    - order_replacement_part: Order parts when diagnostics reveal failures
    
    YOUR PROCESS:
    1. First, run diagnostics to identify the problem
    2. Check warranty status to determine coverage
    3. If hardware failure is confirmed, order replacement parts
    4. Provide clear next steps for the user
    
    YOUR PERSONALITY:
    - Technical but clear (avoid jargon overload)
    - Systematic and thorough in diagnostics
    - Proactive in suggesting solutions
    
    IMPORTANT: You ONLY handle hardware issues. If the user's problem is about
    software or network, politely explain that they need a different specialist.
    """
)

print("✅ Hardware Specialist created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Tools: {len(hardware_specialist.tools)}")

In [ ]:
# Create Software Specialist Agent
software_specialist = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="software_specialist",
    tools=[
        check_license,
        reinstall_software,
        update_application
    ],
    instruction="""
    You are a Software Specialist for TechHelp Solutions IT support.
    
    YOUR EXPERTISE:
    - Software applications (Microsoft Office, Adobe, Zoom, etc.)
    - License management and activation issues
    - Software installation, updates, and troubleshooting
    - Application crashes, freezes, and error messages
    
    YOUR TOOLS:
    - check_license: Verify license status and seat availability
    - reinstall_software: Fix corrupted or misconfigured software
    - update_application: Update software to latest version
    
    YOUR PROCESS:
    1. Check license status first (many issues are license-related)
    2. Try updating the application before reinstalling
    3. Use reinstallation as a last resort for persistent issues
    4. Always verify the fix worked
    
    YOUR PERSONALITY:
    - Patient and methodical
    - Good at explaining technical steps clearly
    - Focused on getting software working properly
    
    IMPORTANT: You ONLY handle software application issues. If the problem is
    hardware or network related, explain that they need a different specialist.
    """
)

print("✅ Software Specialist created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Tools: {len(software_specialist.tools)}")

In [ ]:
# Create Network Specialist Agent
network_specialist = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="network_specialist",
    tools=[
        ping_host,
        check_bandwidth,
        reset_router
    ],
    instruction="""
    You are a Network Specialist for TechHelp Solutions IT support.
    
    YOUR EXPERTISE:
    - Network connectivity (Wi-Fi, ethernet, VPN)
    - Internet access issues
    - Network performance and speed problems
    - Router and network device troubleshooting
    
    YOUR TOOLS:
    - ping_host: Test connectivity to specific hosts or servers
    - check_bandwidth: Measure network speed and performance
    - reset_router: Reboot routers to resolve connectivity issues
    
    YOUR PROCESS:
    1. Use ping to verify basic connectivity first
    2. Check bandwidth if connection is slow but working
    3. Try router reset as a solution for persistent issues
    4. Escalate to network team if problems continue
    
    YOUR PERSONALITY:
    - Quick and efficient
    - Good at diagnosing network issues systematically
    - Focused on restoring connectivity fast
    
    IMPORTANT: You ONLY handle network and connectivity issues. If the problem
    is hardware or software related, explain that they need a different specialist.
    """
)

print("✅ Network Specialist created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Tools: {len(network_specialist.tools)}")

In [ ]:
print("\n" + "="*70)
print("📊 SPECIALIST AGENTS SUMMARY")
print("="*70)
print(f"\n🔧 Hardware Specialist: {hardware_specialist.name}")
print(f"   └─ Tools: check_warranty, run_hardware_diagnostic, order_replacement_part")
print(f"\n💻 Software Specialist: {software_specialist.name}")
print(f"   └─ Tools: check_license, reinstall_software, update_application")
print(f"\n🌐 Network Specialist: {network_specialist.name}")
print(f"   └─ Tools: ping_host, check_bandwidth, reset_router")
print("\n" + "="*70 + "\n")

---

## 🎯 Part 4: Creating the Coordinator Agent

Now comes the magic! The coordinator agent uses the `sub_agents` parameter to manage our specialists.

### How Hierarchical Routing Works

1. **User sends request** → Coordinator receives it
2. **Coordinator analyzes** → "This is a hardware/software/network issue"
3. **Coordinator routes** → Calls `transfer_to_agent(agent_name='specialist')`
4. **Specialist handles** → Uses their specialized tools
5. **User receives response** → From the specialist

### The `sub_agents` Parameter

ADK automatically:
- ✅ Sets parent-child relationships
- ✅ Enables `transfer_to_agent()` function
- ✅ Manages context handoff between agents
- ✅ Ensures each agent only sees relevant information

In [ ]:
# Create Coordinator Agent with sub_agents
coordinator_agent = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="it_support_coordinator",
    sub_agents=[
        hardware_specialist,
        software_specialist,
        network_specialist
    ],
    instruction="""
    You are the IT Support Coordinator for TechHelp Solutions.
    
    YOUR ROLE:
    You are the first point of contact for all IT support requests. Your job is to
    quickly understand the user's issue and route them to the appropriate specialist.
    
    AVAILABLE SPECIALISTS:
    1. hardware_specialist - For physical device issues:
       - Computer won't turn on/boot
       - Hardware failures, strange noises
       - Physical damage, broken parts
       - Printer, monitor, keyboard issues
    
    2. software_specialist - For application issues:
       - Software won't open or crashes
       - License/activation problems
       - Application errors or freezes
       - Need to install/update software
    
    3. network_specialist - For connectivity issues:
       - Can't connect to internet/Wi-Fi
       - Slow network speeds
       - VPN connection problems
       - Network outages
    
    YOUR DECISION PROCESS:
    1. Read the user's issue carefully
    2. Identify keywords: hardware words (device, power, boot, broken) vs
       software words (application, license, install, crash) vs
       network words (wifi, internet, connection, slow)
    3. Use transfer_to_agent() to route to the right specialist
    4. If unclear, ask ONE clarifying question before routing
    
    ROUTING EXAMPLES:
    - "My laptop won't turn on" → hardware_specialist (power issue)
    - "Microsoft Office says my license is invalid" → software_specialist (license)
    - "I can't connect to Wi-Fi" → network_specialist (connectivity)
    - "My computer is running slow" → Ask: "Is it slow opening programs or slow internet?"
    
    YOUR PERSONALITY:
    - Friendly and welcoming
    - Quick to identify the issue type
    - Confident in routing decisions
    
    IMPORTANT:
    - Always route to a specialist - don't try to solve issues yourself
    - Use transfer_to_agent() as soon as you identify the issue type
    - The specialists have the tools and expertise - trust them!
    - Be brief in your initial response before transferring
    """
)

print("✅ Coordinator Agent created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Sub-agents: {len(coordinator_agent.sub_agents)}")
print(f"   Sub-agent names: {[agent.name for agent in coordinator_agent.sub_agents]}")

### Create Runner and Session Service

In [ ]:
# Create session service and app configuration
session_service = InMemorySessionService()
APP_NAME = "it_support_multi_agent"

# Create the runner with coordinator as the root agent
runner = Runner(
    app_name=APP_NAME,
    agent=coordinator_agent,  # Root agent is the coordinator
    session_service=session_service
)

print("✅ Runner initialized!")
print(f"   App Name: {APP_NAME}")
print(f"   Root Agent: {coordinator_agent.name}")
print(f"\n🎯 Multi-Agent System Ready!")
print("   When users send requests, the coordinator will automatically route")
   "   them to the appropriate specialist agent.")

### Helper Function for Multi-Agent Interaction

In [ ]:
# Track created sessions
_created_sessions = set()

async def chat_with_support_async(user_message: str, session_id: str = "session_001", user_id: str = "user_001"):
    """
    Send a message to the IT support multi-agent system (async version).
    The coordinator will route to the appropriate specialist.
    """
    # Create session if needed
    session_key = (session_id, user_id)
    if session_key not in _created_sessions:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=user_id,
            session_id=session_id,
            state={}
        )
        _created_sessions.add(session_key)
    
    content = types.Content(role='user', parts=[types.Part(text=user_message)])
    events = runner.run_async(user_id=user_id, session_id=session_id, new_message=content)
    
    print(f"\n{'='*80}")
    print(f"👤 USER: {user_message}")
    print(f"{'='*80}\n")
    
    current_agent = "coordinator"
    final_response = None
    
    async for event in events:
        # Track which agent is responding (useful for debugging)
        if hasattr(event, 'agent_name') and event.agent_name:
            if event.agent_name != current_agent:
                current_agent = event.agent_name
                print(f"\n🔀 [ROUTING] Transferred to: {current_agent}\n")
        
        if event.is_final_response():
            final_response = event.content.parts[0].text
    
    if final_response:
        # Determine which emoji based on the agent that responded
        agent_emoji = {
            "hardware_specialist": "🔧",
            "software_specialist": "💻",
            "network_specialist": "🌐",
            "it_support_coordinator": "🎯"
        }.get(current_agent, "🤖")
        
        print(f"{agent_emoji} {current_agent.upper().replace('_', ' ')}: {final_response}")
        print(f"\n{'='*80}\n")
        return final_response
    else:
        print("⚠️ No response received.")
        return None

def chat_with_support(user_message: str, session_id: str = "session_001", user_id: str = "user_001"):
    """
    Synchronous wrapper for chat_with_support_async.
    """
    try:
        loop = asyncio.get_running_loop()
        import nest_asyncio
        nest_asyncio.apply()
        return asyncio.run(chat_with_support_async(user_message, session_id, user_id))
    except RuntimeError:
        return asyncio.run(chat_with_support_async(user_message, session_id, user_id))

print("✅ Helper function created!")
print("   This function will show you which agent handles each request.")

---

## 🎬 Part 5: Demonstration - Hierarchical Routing in Action

Let's test our multi-agent system with different types of IT issues. Watch how the coordinator routes each request to the appropriate specialist!

### Demo 1: Hardware Issue (Device Won't Boot)

In [ ]:
chat_with_support(
    "My laptop won't turn on at all. I pressed the power button but nothing happens.",
    session_id="demo_hardware_001"
)

### Demo 2: Software Issue (License Problem)

In [ ]:
chat_with_support(
    "I'm trying to open Microsoft Office but it says my license is invalid.",
    session_id="demo_software_001"
)

### Demo 3: Network Issue (Wi-Fi Connection)

In [ ]:
chat_with_support(
    "I can't connect to the office Wi-Fi. It keeps saying 'connection failed'.",
    session_id="demo_network_001"
)

### Demo 4: Complex Routing (Slow Computer - Ambiguous)

In [ ]:
chat_with_support(
    "My computer is really slow. Everything takes forever.",
    session_id="demo_complex_001"
)

### Demo 5: Printer Hardware Issue

In [ ]:
chat_with_support(
    "The printer is making a weird grinding noise and won't print anything.",
    session_id="demo_hardware_002"
)

### Demo 6: Application Crash (Software Issue)

In [ ]:
chat_with_support(
    "Zoom keeps crashing every time I try to join a meeting.",
    session_id="demo_software_002"
)

---

## 🔍 Part 6: Understanding What Just Happened

### The Routing Decision Process

For each request, the coordinator:

1. **Analyzed** the user's message for keywords and context
2. **Classified** the issue type (hardware / software / network)
3. **Called** `transfer_to_agent(agent_name='specialist')`
4. **Transferred** control to the appropriate specialist
5. **Specialist** used their tools to solve the problem

### Why This Pattern is Powerful

```
Traditional Single Agent:           Multi-Agent Hierarchy:
┌─────────────────────┐            ┌─────────────────────┐
│   One Agent with    │            │    Coordinator      │
│   50+ tools         │            │    (0 tools)        │
│                     │            └──────────┬──────────┘
│ • Overwhelmed       │                       │
│ • Slow responses    │         ┌─────────────┼─────────────┐
│ • Generic answers   │         ▼             ▼             ▼
│ • Hard to maintain  │    ┌─────────┐  ┌─────────┐  ┌─────────┐
└─────────────────────┘    │Hardware │  │Software │  │Network  │
                           │3 tools  │  │3 tools  │  │3 tools  │
                           │Expert   │  │Expert   │  │Expert   │
                           └─────────┘  └─────────┘  └─────────┘
                           
                           • Fast routing
                           • Focused expertise
                           • Easy to scale
```

### Key Observations

1. **Automatic Tool Loading**: Each specialist only loads their 3 tools, not all 9
2. **Context Isolation**: Hardware specialist doesn't see software tool schemas
3. **Seamless Transfer**: User doesn't need to know about routing
4. **Clear Attribution**: We can track which agent handled each request
5. **Independent Updates**: Can modify one specialist without affecting others

### The `transfer_to_agent()` Function

ADK automatically provides this function to the coordinator. It:
- Takes `agent_name` as parameter
- Transfers conversation control to that sub-agent
- Passes relevant context
- Returns control when specialist completes their task

---

## 💡 Part 7: Model Selection Strategy (Discussion)

One powerful feature of multi-agent systems is the ability to use **different models** for different complexity levels.

### This Tutorial's Approach

**We're using `gpt-5-nano` for all agents** to keep costs minimal while learning. This works great because:
- ✅ Routing decisions are usually straightforward (keyword-based)
- ✅ Specialist tasks follow structured processes
- ✅ Most cost-efficient for educational purposes
- ✅ Still demonstrates the hierarchical routing pattern perfectly

### Production Model Selection Patterns

In production systems, you might use different models based on complexity:

```python
# Pattern 1: Smart coordinator, efficient specialists
coordinator = LlmAgent(model=LiteLlm(model="openai/gpt-4o"))         # More capable
specialists = LlmAgent(model=LiteLlm(model="openai/gpt-5-nano"))     # Cost-efficient
# Use Case: Complex routing decisions with many edge cases

# Pattern 2: All same model (what this tutorial uses)
all_agents = LlmAgent(model=LiteLlm(model="openai/gpt-5-nano"))      # Simplest
# Use Case: Clear routing criteria, structured specialist tasks

# Pattern 3: Mix models by task complexity
coordinator = LlmAgent(model=LiteLlm(model="openai/gpt-4o-mini"))    # Mid-tier
routine_specialist = LlmAgent(model=LiteLlm(model="openai/gpt-5-nano"))
complex_specialist = LlmAgent(model=LiteLlm(model="openai/gpt-4o"))
# Use Case: Some specialists handle complex analysis, others do simple lookups

# Pattern 4: Mix by provider (leverage strengths)
coordinator = LlmAgent(model=LiteLlm(model="claude-3-5-sonnet-20241022"))
specialist1 = LlmAgent(model=LiteLlm(model="openai/gpt-5-nano"))
specialist2 = LlmAgent(model=LiteLlm(model="gemini-2.0-flash-exp"))
# Use Case: Different models excel at different tasks
```

### Cost Comparison Example

Assuming 1000 daily support requests:

**Single Agent (gpt-4o for everything):**
- 1000 requests × 1000 tokens × $0.01/1K = **$10/day**

**Multi-Agent (gpt-5-nano for all):**
- Coordinator: 1000 × 200 tokens × $0.0002/1K = **$0.04/day**
- Specialists: 1000 × 800 tokens × $0.0002/1K = **$0.16/day**
- **Total: $0.20/day (98% savings!)**

**Multi-Agent (gpt-4o coordinator + gpt-5-nano specialists):**
- Coordinator: 1000 × 200 tokens × $0.01/1K = **$2/day**
- Specialists: 1000 × 800 tokens × $0.0002/1K = **$0.16/day**
- **Total: $2.16/day (78% savings)**

### When to Upgrade Models

Consider using more powerful models when:

| Scenario | Recommended Upgrade | Why |
|----------|-------------------|------|
| Complex routing logic | Coordinator → gpt-4o | Better at ambiguous cases |
| Specialized analysis | That specialist → gpt-4o | Deeper reasoning needed |
| High-stakes decisions | Critical agents → gpt-4o | Accuracy is paramount |
| Simple lookups | Keep gpt-5-nano | No benefit from upgrade |

### Key Insight

**The multi-agent pattern gives you cost savings REGARDLESS of which models you use**, because:
1. Each specialist loads fewer tools (faster, cheaper)
2. You can mix models (expensive where needed, cheap elsewhere)
3. Specialized instructions make agents more effective
4. Clear separation makes debugging and optimization easier

**For this tutorial, gpt-5-nano is perfect for learning the pattern!**

---

## 🎓 Part 8: Student Exercises

Now it's your turn to practice building multi-agent systems!

### Exercise 1: Add a Security Specialist (Intermediate)

**Task:** Create a 4th specialist for security and access issues.

**Requirements:**
1. Create 2-3 security-related tools:
   - `reset_password(username)`: Reset user password
   - `check_account_status(username)`: Check if account is locked
   - `grant_access(username, resource)`: Grant access to systems
2. Create the `security_specialist` agent with appropriate instructions
3. Add it to the coordinator's `sub_agents` list
4. Test with: "I'm locked out of my account"

**Hint:** Security specialist should handle passwords, account locks, and access permissions.

In [ ]:
# Exercise 1: Your code here

# TODO: Create security tools
def reset_password(username: str) -> Dict[str, any]:
    """
    # YOUR CODE HERE: Implement password reset
    """
    pass

# TODO: Create more security tools

# TODO: Create security_specialist agent
# security_specialist = LlmAgent(...)

# TODO: Update coordinator with new sub-agent
# coordinator_agent = LlmAgent(
#     ...
#     sub_agents=[hardware_specialist, software_specialist, network_specialist, security_specialist]
# )

# TODO: Test it
# chat_with_support("I forgot my password and can't log in", session_id="exercise1")

### Exercise 2: Test Routing Accuracy (Beginner)

**Task:** Test the coordinator with 5 different issue types and verify correct routing.

**Test Cases:**
1. "My monitor has a crack in the screen" → Should go to hardware_specialist
2. "Adobe won't let me save files" → Should go to software_specialist
3. "I can't access the VPN" → Should go to network_specialist
4. "The keyboard keys are stuck" → Should go to hardware_specialist
5. "My email application keeps freezing" → Should go to software_specialist

**Instructions:** Run each test and note which specialist handled it.

In [ ]:
# Exercise 2: Test routing accuracy

test_cases = [
    "My monitor has a crack in the screen",
    "Adobe won't let me save files",
    "I can't access the VPN",
    "The keyboard keys are stuck",
    "My email application keeps freezing"
]

# TODO: Test each case and track which specialist handles it
for i, test_case in enumerate(test_cases, 1):
    print(f"\n{'='*80}")
    print(f"TEST CASE {i}")
    print(f"{'='*80}")
    chat_with_support(test_case, session_id=f"test_{i}")
    # Observe which specialist was called in the output

### Exercise 3: Analyze Session State Isolation (Advanced)

**Task:** Verify that each specialist maintains independent context.

**Test:**
1. In session_1, ask hardware specialist to diagnose a laptop
2. In session_2, ask software specialist about Office license
3. In session_1, ask follow-up about the laptop
4. Verify: Hardware specialist remembers laptop, software specialist doesn't

**Goal:** Understand that specialists only see conversations they're involved in.

In [ ]:
# Exercise 3: Test session isolation

# TODO: Session 1 - Hardware issue
print("\n=== SESSION 1: HARDWARE ISSUE ===")
chat_with_support("My laptop won't boot", session_id="isolation_session_1")

# TODO: Session 2 - Software issue (different session)
print("\n=== SESSION 2: SOFTWARE ISSUE ===")
chat_with_support("Check my Office license", session_id="isolation_session_2")

# TODO: Back to Session 1 - Follow-up
print("\n=== SESSION 1: FOLLOW-UP ===")
chat_with_support("What did the diagnostic say?", session_id="isolation_session_1")

# Observe: Does hardware specialist remember the laptop?
# Observe: Does software specialist know about the laptop? (It shouldn't!)

### Exercise 4: Multi-Model Strategy (Advanced - Optional)

**Task:** Recreate the system with mixed models.

**Instructions:**
1. Use `gpt-4o` for coordinator (better routing decisions)
2. Use `gpt-4o-mini` for specialists (cost-efficient)
3. Compare response quality with all-mini version
4. Estimate cost difference

**Challenge:** Is the improved routing worth the extra cost?

In [ ]:
# Exercise 4: Multi-model setup (Optional)

# TODO: Create coordinator with gpt-4o
# premium_coordinator = LlmAgent(
#     model=LiteLlm(model="openai/gpt-4o"),
#     ...
# )

# TODO: Keep specialists with gpt-4o-mini

# TODO: Test with ambiguous cases to see if routing improves
# Examples of ambiguous cases:
# - "My work is going really slow"
# - "I can't access files"
# - "Something is wrong with my computer"

---

## 🎯 Part 9: Key Takeaways

Congratulations! You've mastered the most important design pattern in production AI systems!

### What You Learned Today ✅

1. **Multi-Agent Architecture**
   - Coordinator + specialist pattern (hierarchical routing)
   - How to use `sub_agents` parameter
   - Agent communication via `transfer_to_agent()`

2. **Benefits of Specialization**
   - Faster responses (fewer tools per agent)
   - Better quality (focused expertise)
   - Easier maintenance (isolated changes)
   - Cost optimization (right model for right task)

3. **Production Patterns**
   - Separation of concerns (domain-specific agents)
   - Context isolation (agents only see relevant info)
   - Scalability (easy to add new specialists)
   - Debugging (clear attribution of responses)

4. **ADK Multi-Agent Features**
   - `sub_agents` parameter for hierarchy
   - Automatic `transfer_to_agent()` function
   - Parent-child relationship management
   - Model-agnostic (works with any LLM provider)

### Design Principles for Multi-Agent Systems 🏗️

1. **Single Responsibility**: Each agent has one clear domain
2. **Minimal Tools**: Specialists have 3-5 focused tools max
3. **Clear Routing**: Coordinator has obvious decision criteria
4. **Context Isolation**: Agents only see what they need
5. **Graceful Fallback**: Handle unclear cases with clarifying questions

### When to Use This Pattern 📊

**✅ Use Multi-Agent When:**
- You have 10+ tools across different domains
- Issues fall into clear categories
- Different expertise levels are needed
- You want to optimize costs
- System needs to scale over time

**❌ Single Agent is Fine When:**
- You have < 5 tools total
- All tools are closely related
- Routing overhead isn't worth it
- System is small and won't grow

### Real-World Applications 🌍

This exact pattern powers:
- **Customer Service**: Route by department (sales, support, billing)
- **Healthcare**: Route by specialty (general, specialist, pharmacy)
- **Finance**: Route by function (trading, research, compliance)
- **E-commerce**: Route by topic (products, orders, returns)
- **HR Systems**: Route by category (payroll, benefits, recruitment)

### What's Next 🚀

Future lessons will cover:
- **Advanced State Management**: Persistent sessions with Firestore
- **Agent-to-Agent Communication**: Specialists collaborating
- **Dynamic Agent Creation**: Creating specialists on-the-fly
- **MCP Integration**: Connecting to external tool servers
- **Production Deployment**: Scaling multi-agent systems

### Resources 📚

- [ADK Multi-Agent Documentation](https://google.github.io/adk-docs/agents/multi-agents/)
- [ADK Sub-Agents Guide](https://google.github.io/adk-docs/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [LiteLLM Multi-Provider](https://docs.litellm.ai/docs/)

---

### 🎓 Final Challenge

Build your own multi-agent system for a domain you care about:
- 1 coordinator agent
- 3+ specialist agents with unique tools
- Test with 10+ realistic scenarios
- Measure routing accuracy

**You now know the #1 pattern for building production AI systems! 🎉**

---